In [59]:
import pandas as pd
from Bio.Seq import Seq
import copy

In [60]:
        df = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/kemai/20220517merTRflilter/sc-1/03.assemble/assemble/sc-1_contig.csv', sep='\t', header=None)
        df.columns = ['barcode', 'is_cell', 'contig_id', 'high_confidence', 'length', 'chain', 'v_gene', 'd_gene', 'j_gene', 'c_gene', 'full_length', 'productive', 'cdr3', 'cdr3_nt', 'reads', 'umis']
        df['d_gene'] = df['d_gene'].apply(lambda x: x.split('(')[0] if not x == '*' else 'None')
        df['c_gene'] = df['c_gene'].apply(lambda x: x.split('(')[0] if not x == '*' else 'None')
        df['cdr3'] = df['cdr3_nt'].apply(lambda x: 'None' if "*" in str(Seq(x).translate()) or not len(x)%3==0 else str(Seq(x).translate()))
        df['productive'] = df['cdr3'].apply(lambda x: False if x=='None' else True)

/Personal/chenjunjie/.local/lib/python3.7/site-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [62]:
df.sort_values(by='umis', ascending=False, inplace=True)

In [63]:
        #df_chain_heavy = df[df['chain'] == 'TRA']
        #df_chain_light = df[df['chain'] == 'TRB']
        df_chain_heavy = df[df['chain']=='IGH']
        df_chain_light = df[(df['chain']=='IGK') | (df['chain']=='IGL')]
        df_chain_heavy = df_chain_heavy.drop_duplicates(['barcode'])
        df_chain_light = df_chain_light.drop_duplicates(['barcode'])
        df_for_clono = pd.concat([df_chain_heavy, df_chain_light], ignore_index=True)

In [ ]:
        trust_report = pd.read_csv('/SGRNJ06/randd/USER/cjj/TESTDATA/test_trust/20220530/fltest/03.assemble/assemble//trust_filter_report.out', sep='\t')
        correct_cdr3 = set(df_for_clono['cdr3']).intersection(set(trust_report.CDR3aa))
        correct_cdr3 = [i for i in correct_cdr3 if i.startswith('C')]
        correct_cdr3 = [i for i in correct_cdr3 if len(i)>=5]
        correct_cdr3 = [i for i in correct_cdr3 if 'UAG' or 'UAA' or 'UGA' not in i]
        df_for_clono = df_for_clono[df_for_clono['cdr3'].isin(correct_cdr3)]

In [ ]:
def reversed_compl(seq):
    return str(Seq(seq).reverse_complement()) 

In [ ]:
        assign_info = pd.read_csv('/SGRNJ06/randd/USER/cjj/TESTDATA/test_trust/20220530/fltest/04.summarize/fltest_assign.csv')
        assign_info.rename(columns={'Unnamed: 0':'barcode'},inplace=True)

        bg_barcode = set(assign_info[assign_info['celltype']!='T_cells'].barcode)
        bg_barcode = {reversed_compl(i) for i in bg_barcode}

        df_assign = copy.deepcopy(df_for_clono)
        df_assign.loc[~df_assign['barcode'].isin(bg_barcode),'umis'] = df_assign.loc[~df_assign['barcode'].isin(bg_barcode),'umis'].apply(lambda x:x*3)
        df_assign.loc[df_assign['barcode'].isin(bg_barcode),'umis'] = df_assign.loc[df_assign['barcode'].isin(bg_barcode),'umis'].apply(lambda x:x//3)

In [ ]:
assign_info.groupby('celltype').count()

In [ ]:
trust_report

In [64]:
df_for_clono

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis
0,TGATACGTTCAGGAGGTTGTTCCA,True,TGATACGTTCAGGAGGTTGTTCCA_899,True,517,IGH,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,IGHG2B*03,True,True,CARGITGIDYW,TGTGCGAGGGGTATAACTGGAATTGACTACTGG,242751,105937
1,TAGACGGAGTCGCTATTCGAGCGT,True,TAGACGGAGTCGCTATTCGAGCGT_1286,True,605,IGH,IGHV3-23*04,IGHD7-27*01,IGHJ2*01,IGHG1*01,True,True,CAKDRNWGYEYFDLW,TGTGCGAAAGATCGAAACTGGGGATACGAGTACTTCGATCTCTGG,146639,79062
2,GTTGTCGGTCTCGGTTTGCATAGT,True,GTTGTCGGTCTCGGTTTGCATAGT_525,True,804,IGH,IGHV3-20*01,IGHD3-9*01,IGHJ4*02,IGHG2B*03,True,True,CARDLTLFDYW,TGTGCGAGAGATCTAACTCTTTTTGACTACTGG,138706,71297
3,TTCTGTGTGGTCGTGTCGATGTTT,True,TTCTGTGTGGTCGTGTCGATGTTT_5973,True,522,IGH,IGHV4-39*01,IGHD1-7*01,IGHJ2*01,IGHA*09,True,True,CASLYNWNYDWFDPWGLW,TGTGCGAGCCTGTATAACTGGAACTACGACTGGTTCGACCCCTGGG...,74544,45872
4,GTGTGTCGGCTAACTCTCGAAGTG,True,GTGTGTCGGCTAACTCTCGAAGTG_2191,True,536,IGH,IGHV3-48*03,IGHD3-22*01,IGHJ4*02,IGHA*09,True,True,CAREADWLVHYYFDYW,TGTGCGAGAGAGGCGGACTGGCTGGTTCATTACTACTTTGACTACTGG,55476,39029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,GACCTTAGGTCTTGGCGAGGATGG,True,GACCTTAGGTCTTGGCGAGGATGG_4323,True,499,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,32,28
12140,GCACTGTCTTCGCACCTACTTCGG,True,GCACTGTCTTCGCACCTACTTCGG_4316,True,566,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,30,27
12141,GGCTACAGTGGCTCAGGTGTCCTT,True,GGCTACAGTGGCTCAGGTGTCCTT_4312,True,520,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,34,26
12142,TCATTGAGGGATTAGGTGCGATCT,True,TCATTGAGGGATTAGGTGCGATCT_4308,True,510,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,26,22


In [ ]:
barcode_report = pd.read_csv('/SGRNJ06/randd/USER/cjj/TESTDATA/test_trust/20220530/fltest/03.assemble/assemble/barcoderepfl.tsv', sep='\t')

In [ ]:
barcode_report['#barcode']

In [66]:
import numpy as np

In [67]:
df_chain_heavy

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis
46061,TGATACGTTCAGGAGGTTGTTCCA,True,TGATACGTTCAGGAGGTTGTTCCA_899,True,517,IGH,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,IGHG2B*03,True,True,CARGITGIDYW,TGTGCGAGGGGTATAACTGGAATTGACTACTGG,242751,105937
24729,TAGACGGAGTCGCTATTCGAGCGT,True,TAGACGGAGTCGCTATTCGAGCGT_1286,True,605,IGH,IGHV3-23*04,IGHD7-27*01,IGHJ2*01,IGHG1*01,True,True,CAKDRNWGYEYFDLW,TGTGCGAAAGATCGAAACTGGGGATACGAGTACTTCGATCTCTGG,146639,79062
45767,GTTGTCGGTCTCGGTTTGCATAGT,True,GTTGTCGGTCTCGGTTTGCATAGT_525,True,804,IGH,IGHV3-20*01,IGHD3-9*01,IGHJ4*02,IGHG2B*03,True,True,CARDLTLFDYW,TGTGCGAGAGATCTAACTCTTTTTGACTACTGG,138706,71297
36432,TTCTGTGTGGTCGTGTCGATGTTT,True,TTCTGTGTGGTCGTGTCGATGTTT_5973,True,522,IGH,IGHV4-39*01,IGHD1-7*01,IGHJ2*01,IGHA*09,True,True,CASLYNWNYDWFDPWGLW,TGTGCGAGCCTGTATAACTGGAACTACGACTGGTTCGACCCCTGGG...,74544,45872
33185,GTGTGTCGGCTAACTCTCGAAGTG,True,GTGTGTCGGCTAACTCTCGAAGTG_2191,True,536,IGH,IGHV3-48*03,IGHD3-22*01,IGHJ4*02,IGHA*09,True,True,CAREADWLVHYYFDYW,TGTGCGAGAGAGGCGGACTGGCTGGTTCATTACTACTTTGACTACTGG,55476,39029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11180,GTGTCCTTTAGTGACTGAATCTGT,True,GTGTCCTTTAGTGACTGAATCTGT_21499,True,506,IGH,IGHV4-39*07,IGHD4-11*01,IGHJ6*02,IGHG2B*03,True,True,CARGWDYSNYDYYYYYGMDVW,TGTGCGAGAGGTTGGGACTACAGTAACTACGATTACTACTACTACT...,0,0
11265,TTACTCGCTCTCACGGTAAGTTCG,True,TTACTCGCTCTCACGGTAAGTTCG_21683,True,400,IGH,IGHV4-39*01,IGHD6-13*01,IGHJ5*02,IGHG1*01,True,True,CARHSSSWEGWFDPW,TGTGCGAGACATAGCAGCAGCTGGGAGGGGTGGTTCGACCCCTGG,0,0
12980,TCCTCAATTACAGGATTGTACCTT,True,TCCTCAATTACAGGATTGTACCTT_26654,True,482,IGH,IGHV4-59*04,IGHD6-13*01,IGHJ5*02,IGHG1*01,True,True,CARHSSSWEGWFDPW,TGTGCGAGACATAGCAGCAGCTGGGAGGGGTGGTTCGACCCCTGG,0,0
17744,GTTGTCGGTAGACGGATCCTCAAT,True,GTTGTCGGTAGACGGATCCTCAAT_43784,True,461,IGH,IGHV1-18*01,IGHD1-14*01,IGHJ3*02,IGHA*09,True,True,CARKNNWNHGGFDIW,TGTGCGAGAAAGAATAACTGGAACCACGGTGGTTTTGATATCTGG,0,0


In [68]:
df_chain_light

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis
45511,TGATACGTTCAGGAGGTTGTTCCA,True,TGATACGTTCAGGAGGTTGTTCCA_3,True,583,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,5542794,259614
24264,TAGACGGAGTCGCTATTCGAGCGT,True,TAGACGGAGTCGCTATTCGAGCGT_284,True,543,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,3711159,255004
189,TGAAGCCATGTCTATCTAAGTTCG,True,TGAAGCCATGTCTATCTAAGTTCG_189,True,561,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,2162312,246006
32245,GTGTGTCGGCTAACTCTCGAAGTG,True,GTGTGTCGGCTAACTCTCGAAGTG_177,True,528,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,2208425,245880
45640,GTTGTCGGTCTCGGTTTGCATAGT,True,GTTGTCGGTCTCGGTTTGCATAGT_142,True,585,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,435280,163043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4323,GACCTTAGGTCTTGGCGAGGATGG,True,GACCTTAGGTCTTGGCGAGGATGG_4323,True,499,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,32,28
4316,GCACTGTCTTCGCACCTACTTCGG,True,GCACTGTCTTCGCACCTACTTCGG_4316,True,566,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,30,27
4312,GGCTACAGTGGCTCAGGTGTCCTT,True,GGCTACAGTGGCTCAGGTGTCCTT_4312,True,520,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,34,26
4308,TCATTGAGGGATTAGGTGCGATCT,True,TCATTGAGGGATTAGGTGCGATCT_4308,True,510,IGK,IGKV3-11*01,None,IGKJ1*01,IGKC*03,True,True,CQQRSNWPPTF,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,26,22


In [69]:
class Auto():
    """
    threshold = top {percentile}% cell count / coef
    count is usually UMI count.
    >>> array = [50] * 100 + [30] * 100 + [10] * 100 + [4] * 100
    >>> Auto(array, coef=10).run()
    5
    >>> Auto(array, percentile=70, coef=3).run()
    10
    >>> Auto(array, percentile=50, coef=10, expected_cell_num=100).run()
    5
    >>> Auto([1, 2, 20, 30, 40], expected_cell_num=4, percentile=50, coef=10).run()
    2
    """
    def __init__(self, array, percentile=99, coef=3, expected_cell_num=None, **kwargs):
        self.array = [x for x in array if x > 0 ]
        self.percentile = percentile
        self.coef = int(coef)
        self.expected_cell_num = expected_cell_num
        self.kwargs = kwargs
    
    def run(self):
        array = self.array
        if not array:
            return 1

        if not self.expected_cell_num:
            expected_cell_num = len(array)
        else:
            expected_cell_num = self.expected_cell_num
            if expected_cell_num > len(array):
                print('Warning: expected_cell_num > len(array)')
                expected_cell_num = len(array)
                      
        sorted_counts = sorted(array, reverse=True)
        count_cell_percentile = np.percentile(sorted_counts[:expected_cell_num], self.percentile)
        threshold = int(count_cell_percentile / self.coef)

        return threshold

In [70]:
def target_cell_calling(df_UMI_sum, expected_target_cell_num=3000, target_barcodes=None, weight=6, coef=5, 
    percentile=80, umi_col='umis'):
    """
    Args:
        df_UMI_sum: A dataframe with columns highest umi's contig and UMI.
    
    Returns:
        umi_threhold: int
        target_cell_barcodes: list
    >>> df_UMI_sum = pd.DataFrame({"contig_id": ["A", "B", "C", "D", "E"], "UMI": [1, 2, 1, 30, 40]})
    >>> umi_threshold, target_cell_barcodes = target_cell_calling(df_UMI_sum, expected_target_cell_num=5, percentile=80, coef=5, target_barcodes=["A", "C"])
    >>> umi_threshold == 3
    True
    >>> target_cell_barcodes == {'A', 'C', 'D', 'E'}
    True
    """
    target_barcodes = {Summarize.reversed_compl(i) for i in target_barcodes}
    umi_threshold = Auto(list(df_UMI_sum[umi_col]), expected_cell_num=expected_target_cell_num, coef=coef, percentile=percentile).run()

    # avoid change the original dataframe
    df_temp = df_UMI_sum.copy()
    if target_barcodes:
        df_temp[umi_col] = df_temp.apply(
            lambda row:  row[umi_col] * weight if row['barcode'] in target_barcodes else row[umi_col], axis=1)
             
    target_contigs = set(df_temp.loc[df_temp[umi_col] >= umi_threshold].contig_id)

    return target_contigs

In [ ]:
target_contigs = target_cell_calling(
            df_chain_light, 
            expected_target_cell_num=3000, 
            target_barcodes=self.target_barcodes,
            )